It turns out for CIFAR-10, augmentation does not help noticably. The gap with Fortran is due to the training, not the augmentation!


In [1]:
#!/usr/bin/env python3
"""
CIFAR-10 Baseline Test - NO AUGMENTATION
========================================
Modified from cifar10-7.ipynb to remove all data augmentation
This will establish the Python baseline accuracy without augmentation
for direct comparison with Fortran implementation.
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import time
import sys
import os
import matplotlib.pyplot as plt

# Import version 7 TensorCore
try:
    from tensor_core_wrapper7 import TensorCore, TensorCoreLinear, get_global_tensorcore
    print("✅ Imported tensor_core_wrapper7 with TensorCoreLinear")
    TENSORCORE_AVAILABLE = True
except ImportError:
    print("❌ Could not import tensor_core_wrapper7!")
    print("   Please ensure tensor_core_wrapper7 is available")
    sys.exit(1)

# CIFAR-10 classes
CIFAR10_CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 
                   'dog', 'frog', 'horse', 'ship', 'truck')

def set_debug_mode(enabled):
    """Set debug mode for TensorCore operations"""
    try:
        tc = TensorCore(debug_mode=enabled)
        print(f"🔧 TensorCore debug mode: {'enabled' if enabled else 'disabled'}")
    except Exception as e:
        print(f"⚠️ Could not set debug mode: {e}")

def load_cifar10_data_no_augmentation(batch_size=128, num_workers=4):
    """
    Load CIFAR-10 with NO AUGMENTATION - only normalization
    """
    print("📁 Loading CIFAR-10 dataset with NO AUGMENTATION...")
    
    # NO AUGMENTATION - only normalization for both train and test
    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
    
    # Same transform for test (no augmentation needed anyway)
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
    
    # Load datasets
    train_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=train_transform
    )
    test_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=test_transform
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, 
        num_workers=num_workers, pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, 
        num_workers=num_workers, pin_memory=True
    )
    
    print(f"✅ CIFAR-10 loaded with NO AUGMENTATION:")
    print(f"   Training samples: {len(train_dataset):,}")
    print(f"   Test samples: {len(test_dataset):,}")
    print(f"   Batch size: {batch_size}")
    print(f"   Only normalization applied: (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)")
    
    return train_loader, test_loader

# =============================================================================
# MODEL ARCHITECTURE - Same as Fortran version
# =============================================================================

class CIFAR10NetTensorCoreBaseline(nn.Module):
    """
    CNN model matching the Fortran architecture exactly
    No augmentation dependency - baseline comparison
    """
    def __init__(self):
        super(CIFAR10NetTensorCoreBaseline, self).__init__()
        
        # Convolutional layers (matching Fortran)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Pooling
        self.pool = nn.MaxPool2d(2, 2)
        
        # Calculate feature size after conv layers: 32->16->8->4, 4*4*128 = 2048
        self.feature_size = 4 * 4 * 128
        
        # Fully connected layers using TensorCore
        self.fc1 = TensorCoreLinear(self.feature_size, 512)
        self.fc2 = TensorCoreLinear(512, 256)  
        self.fc3 = TensorCoreLinear(256, 10)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # Conv block 1: Conv -> BN -> LeakyReLU -> Pool
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.leaky_relu(x, negative_slope=0.01)
        x = self.pool(x)  # 32x32 -> 16x16
        
        # Conv block 2: Conv -> BN -> LeakyReLU -> Pool  
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.leaky_relu(x, negative_slope=0.01)
        x = self.pool(x)  # 16x16 -> 8x8
        
        # Conv block 3: Conv -> BN -> LeakyReLU -> Pool
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.leaky_relu(x, negative_slope=0.01)
        x = self.pool(x)  # 8x8 -> 4x4
        
        # Flatten and FC layers
        x = torch.flatten(x, 1)  # Shape: (batch_size, 2048)
        
        x = self.fc1(x)
        x = F.leaky_relu(x, negative_slope=0.01)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = F.leaky_relu(x, negative_slope=0.01)
        x = self.dropout(x)
        
        x = self.fc3(x)
        return x

# =============================================================================
# TRAINING FUNCTIONS
# =============================================================================

def train_epoch(model, train_loader, optimizer, criterion, device, epoch):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
        
        # Progress update every 100 batches
        if batch_idx % 100 == 0:
            print(f'   Batch {batch_idx:3d}/{len(train_loader)} '
                  f'Loss: {loss.item():.6f} '
                  f'Acc: {100.*correct/total:.3f}%')
    
    epoch_time = time.time() - start_time
    avg_loss = running_loss / len(train_loader)
    accuracy = 100. * correct / total
    
    return avg_loss, accuracy, epoch_time

def evaluate_model(model, test_loader, criterion, device):
    """Evaluate model on test set"""
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    class_correct = [0] * 10
    class_total = [0] * 10
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
            # Per-class accuracy
            c = (predicted == target).squeeze()
            for i in range(target.size(0)):
                label = target[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    
    test_loss /= len(test_loader)
    accuracy = 100. * correct / total
    
    # Calculate per-class accuracies
    class_accuracies = []
    for i in range(10):
        if class_total[i] > 0:
            class_acc = 100. * class_correct[i] / class_total[i]
            class_accuracies.append(class_acc)
            print(f'   {CIFAR10_CLASSES[i]:>6}: {class_acc:6.2f}%')
        else:
            class_accuracies.append(0.0)
    
    return test_loss, accuracy, 0, class_accuracies  # 0 for eval_time compatibility

# =============================================================================
# MAIN TRAINING LOOP
# =============================================================================

def main_baseline_training(num_epochs=15, batch_size=128, lr=0.001):
    """
    Main training function - NO AUGMENTATION BASELINE
    """
    print("🚀 CIFAR-10 BASELINE TRAINING - NO AUGMENTATION")
    print("=" * 60)
    
    # Device setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    
    # Disable TensorCore debug for speed
    set_debug_mode(False)
    
    # Load data WITHOUT augmentation
    train_loader, test_loader = load_cifar10_data_no_augmentation(batch_size=batch_size)
    
    # Initialize model (same architecture as successful version 6)
    torch.manual_seed(42)  # Reproducibility
    model = CIFAR10NetTensorCoreBaseline().to(device)
    
    # Count parameters
    params = sum(p.numel() for p in model.parameters())
    print(f"\n📊 Model Information:")
    print(f"   Parameters: {params:,}")
    print(f"   Epochs: {num_epochs}")
    print(f"   Batch size: {batch_size}")
    print(f"   Learning rate: {lr}")
    print(f"   Architecture: Same as Fortran version")
    print(f"   Augmentation: NONE (baseline test)")
    
    # Optimizer and scheduler (matching Python version but no augmentation)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    criterion = nn.CrossEntropyLoss()
    
    # Training history
    history = {
        'train_loss': [], 'train_acc': [], 'test_loss': [], 
        'test_acc': [], 'epoch_times': []
    }
    
    best_test_acc = 0.0
    total_time = 0.0
    
    print(f"\n🏋️ Starting Baseline Training (NO AUGMENTATION)...")
    
    for epoch in range(1, num_epochs + 1):
        print(f"\n📈 EPOCH {epoch}/{num_epochs}")
        print("-" * 60)
        
        # Training
        train_loss, train_acc, epoch_time = train_epoch(
            model, train_loader, optimizer, criterion, device, epoch
        )
        
        # Evaluation
        test_loss, test_acc, eval_time, class_accs = evaluate_model(
            model, test_loader, criterion, device
        )
        
        # Learning rate step
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        print(f"📉 Learning rate: {current_lr:.6f}")
        
        # Track best performance
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            print(f"💾 New best accuracy: {best_test_acc:.2f}%")
        
        # Update history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        history['epoch_times'].append(epoch_time)
        
        total_time += epoch_time
        
        print("-" * 60)
    
    # Final results
    print(f"\n🎉 BASELINE Training Completed (NO AUGMENTATION)!")
    print("=" * 60)
    print(f"🏆 Best test accuracy: {best_test_acc:.2f}%")
    print(f"⏱️ Total training time: {total_time:.1f}s")
    print(f"📊 Average time per epoch: {total_time/num_epochs:.1f}s")
    
    # Compare with expected results
    print(f"\n📊 Baseline Performance Analysis:")
    print(f"   Python baseline (no aug): {best_test_acc:.2f}%")
    print(f"   Fortran current (no aug): ~58.7%")
    print(f"   Gap to close: {abs(best_test_acc - 58.7):.1f} percentage points")
    
    if best_test_acc > 65.0:
        print("✅ GOOD: Baseline shows Python can achieve higher accuracy")
        print("   The gap is likely in training implementation, not augmentation")
    elif best_test_acc < 60.0:
        print("⚠️ CONCERNING: Python baseline is also low")
        print("   This suggests model architecture or hyperparameter issues")
    else:
        print("📊 COMPARABLE: Python and Fortran baselines are similar")
        print("   Focus should be on augmentation implementation")
    
    return history, best_test_acc

if __name__ == "__main__":
    # Run baseline test
    print("🧪 Running CIFAR-10 Baseline Test (NO AUGMENTATION)")
    print("This will establish the Python baseline for comparison")
    print()
    
    history, best_acc = main_baseline_training(num_epochs=15, batch_size=128, lr=0.001)
    
    print(f"\n📋 Summary:")
    print(f"   Python baseline (no augmentation): {best_acc:.2f}%")
    print(f"   Use this to compare with Fortran's 58.7% no-augmentation result")
    print(f"   Any remaining gap indicates fundamental training differences")
# works with py313 but crashes with py314!
# needs tensor_core_gemm7b.cuf to work (7c and 7d have errors)

✅ Imported tensor_core_wrapper7 with TensorCoreLinear
🧪 Running CIFAR-10 Baseline Test (NO AUGMENTATION)
This will establish the Python baseline for comparison

🚀 CIFAR-10 BASELINE TRAINING - NO AUGMENTATION
Device: cuda
🔧 TensorCore debug mode: disabled
📁 Loading CIFAR-10 dataset with NO AUGMENTATION...
✅ CIFAR-10 loaded with NO AUGMENTATION:
   Training samples: 50,000
   Test samples: 10,000
   Batch size: 128
   Only normalization applied: (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)

📊 Model Information:
   Parameters: 1,276,682
   Epochs: 15
   Batch size: 128
   Learning rate: 0.001
   Architecture: Same as Fortran version
   Augmentation: NONE (baseline test)

🏋️ Starting Baseline Training (NO AUGMENTATION)...

📈 EPOCH 1/15
------------------------------------------------------------
   Batch   0/391 Loss: 2.340947 Acc: 11.719%
   Batch 100/391 Loss: 1.478746 Acc: 32.534%
   Batch 200/391 Loss: 1.343262 Acc: 39.342%
   Batch 300/391 Loss: 1.418553 Acc: 43.628%
    plane: 

In [2]:
#!/usr/bin/env python3
"""
Python cuDNN BatchNorm Reference Implementation
Compare exact cuDNN batch normalization behavior vs your Fortran implementation
"""

import torch
import torch.nn as nn
import numpy as np
import json
import os

def create_test_data():
    """Create identical test data for both Python and Fortran"""
    
    # Set fixed seed for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    
    # Test parameters matching your Fortran setup
    batch_size = 128
    channels = 32
    height = 32 
    width = 32
    
    # Create test input (same as CIFAR-10 dimensions after first conv)
    input_data = torch.randn(batch_size, channels, height, width, dtype=torch.float64).cuda()
    
    # Create BatchNorm layer with exact PyTorch defaults
    bn_layer = nn.BatchNorm2d(
        num_features=channels,
        eps=1e-05,           # PyTorch default
        momentum=0.1,        # PyTorch default  
        affine=True,         # learnable parameters
        track_running_stats=True
    ).cuda().double()  # Use double precision to match Fortran
    
    return input_data, bn_layer

def extract_batchnorm_parameters(bn_layer):
    """Extract all BatchNorm parameters for comparison"""
    
    params = {
        'weight': bn_layer.weight.detach().cpu().numpy(),
        'bias': bn_layer.bias.detach().cpu().numpy(),
        'running_mean': bn_layer.running_mean.detach().cpu().numpy(),
        'running_var': bn_layer.running_var.detach().cpu().numpy(),
        'eps': bn_layer.eps,
        'momentum': bn_layer.momentum,
        'num_batches_tracked': bn_layer.num_batches_tracked.item()
    }
    
    return params

def test_training_mode_batchnorm():
    """Test BatchNorm in training mode (updates running statistics)"""
    
    print("🔬 Testing BatchNorm in TRAINING mode")
    print("=" * 50)
    
    input_data, bn_layer = create_test_data()
    
    # Ensure training mode
    bn_layer.train()
    
    print(f"📊 Input shape: {input_data.shape}")
    print(f"📊 Input mean: {input_data.mean().item():.6f}")
    print(f"📊 Input std: {input_data.std().item():.6f}")
    print()
    
    # Get initial parameters
    print("📋 INITIAL BatchNorm Parameters:")
    initial_params = extract_batchnorm_parameters(bn_layer)
    print(f"   Weight (gamma) - first 5: {initial_params['weight'][:5]}")
    print(f"   Bias (beta) - first 5: {initial_params['bias'][:5]}")
    print(f"   Running mean - first 5: {initial_params['running_mean'][:5]}")
    print(f"   Running var - first 5: {initial_params['running_var'][:5]}")
    print(f"   Momentum: {initial_params['momentum']}")
    print(f"   Epsilon: {initial_params['eps']}")
    print()
    
    # Forward pass
    print("🔄 Performing forward pass...")
    output = bn_layer(input_data)
    
    print(f"📊 Output shape: {output.shape}")
    print(f"📊 Output mean: {output.mean().item():.6f}")
    print(f"📊 Output std: {output.std().item():.6f}")
    print()
    
    # Get updated parameters
    print("📋 UPDATED BatchNorm Parameters (after training forward pass):")
    updated_params = extract_batchnorm_parameters(bn_layer)
    print(f"   Running mean - first 5: {updated_params['running_mean'][:5]}")
    print(f"   Running var - first 5: {updated_params['running_var'][:5]}")
    print()
    
    # Compute batch statistics for comparison
    batch_mean = input_data.mean(dim=[0, 2, 3])  # Average over batch, height, width
    batch_var = input_data.var(dim=[0, 2, 3], unbiased=False)  # Biased variance
    
    print("📋 COMPUTED Batch Statistics (for verification):")
    print(f"   Batch mean - first 5: {batch_mean[:5].cpu().numpy()}")
    print(f"   Batch var - first 5: {batch_var[:5].cpu().numpy()}")
    print()
    
    # Verify the momentum update formula
    expected_running_mean = (1 - initial_params['momentum']) * initial_params['running_mean'] + \
                           initial_params['momentum'] * batch_mean.cpu().numpy()
    expected_running_var = (1 - initial_params['momentum']) * initial_params['running_var'] + \
                          initial_params['momentum'] * batch_var.cpu().numpy()
    
    print("🔍 MOMENTUM FORMULA VERIFICATION:")
    print(f"   Expected running mean - first 5: {expected_running_mean[:5]}")
    print(f"   Actual running mean - first 5: {updated_params['running_mean'][:5]}")
    print(f"   Match: {np.allclose(expected_running_mean[:5], updated_params['running_mean'][:5], atol=1e-6)}")
    print()
    print(f"   Expected running var - first 5: {expected_running_var[:5]}")
    print(f"   Actual running var - first 5: {updated_params['running_var'][:5]}")
    print(f"   Match: {np.allclose(expected_running_var[:5], updated_params['running_var'][:5], atol=1e-6)}")
    print()
    
    return {
        'input': input_data.detach().cpu().numpy(),
        'output': output.detach().cpu().numpy(),
        'initial_params': initial_params,
        'updated_params': updated_params,
        'batch_mean': batch_mean.detach().cpu().numpy(),
        'batch_var': batch_var.detach().cpu().numpy()
    }

def test_inference_mode_batchnorm():
    """Test BatchNorm in inference mode (uses running statistics)"""
    
    print("🔬 Testing BatchNorm in INFERENCE mode")
    print("=" * 50)
    
    input_data, bn_layer = create_test_data()
    
    # First, do a training pass to populate running statistics
    bn_layer.train()
    _ = bn_layer(input_data)
    
    # Now switch to eval mode
    bn_layer.eval()
    
    # Get running statistics before inference
    before_params = extract_batchnorm_parameters(bn_layer)
    print("📋 Running Statistics BEFORE inference:")
    print(f"   Running mean - first 5: {before_params['running_mean'][:5]}")
    print(f"   Running var - first 5: {before_params['running_var'][:5]}")
    print()
    
    # Forward pass in inference mode
    output = bn_layer(input_data)
    
    # Get running statistics after inference
    after_params = extract_batchnorm_parameters(bn_layer)
    print("📋 Running Statistics AFTER inference:")
    print(f"   Running mean - first 5: {after_params['running_mean'][:5]}")
    print(f"   Running var - first 5: {after_params['running_var'][:5]}")
    print()
    
    # Verify they didn't change
    mean_unchanged = np.allclose(before_params['running_mean'], after_params['running_mean'])
    var_unchanged = np.allclose(before_params['running_var'], after_params['running_var'])
    
    print(f"✅ Running mean unchanged: {mean_unchanged}")
    print(f"✅ Running var unchanged: {var_unchanged}")
    print()
    
    return {
        'input': input_data.detach().cpu().numpy(),
        'output': output.detach().cpu().numpy(),
        'running_stats': after_params
    }

def save_reference_data(training_result, inference_result):
    """Save reference data for Fortran comparison"""
    
    reference_data = {
        'training_mode': {
            'input_shape': training_result['input'].shape,
            'input_sample': training_result['input'][:2, :5, :5, :5].tolist(),  # Small sample for verification
            'output_sample': training_result['output'][:2, :5, :5, :5].tolist(),
            'batch_statistics': {
                'mean': training_result['batch_mean'].tolist(),
                'var': training_result['batch_var'].tolist()
            },
            'running_statistics_before': {
                'mean': training_result['initial_params']['running_mean'].tolist(),
                'var': training_result['initial_params']['running_var'].tolist()
            },
            'running_statistics_after': {
                'mean': training_result['updated_params']['running_mean'].tolist(),
                'var': training_result['updated_params']['running_var'].tolist()
            },
            'parameters': {
                'weight': training_result['initial_params']['weight'].tolist(),
                'bias': training_result['initial_params']['bias'].tolist(),
                'eps': training_result['initial_params']['eps'],
                'momentum': training_result['initial_params']['momentum']
            }
        },
        'inference_mode': {
            'input_sample': inference_result['input'][:2, :5, :5, :5].tolist(),
            'output_sample': inference_result['output'][:2, :5, :5, :5].tolist(),
            'running_statistics': {
                'mean': inference_result['running_stats']['running_mean'].tolist(),
                'var': inference_result['running_stats']['running_var'].tolist()
            }
        }
    }
    
    with open('batchnorm_reference_data.json', 'w') as f:
        json.dump(reference_data, f, indent=2)
    
    print("💾 Reference data saved to 'batchnorm_reference_data.json'")
    print("   Use this file to compare with your Fortran implementation!")

def main():
    """Main test function"""
    
    print("🚀 Python cuDNN BatchNorm Reference Implementation")
    print("=" * 60)
    print()
    
    # Test training mode
    training_result = test_training_mode_batchnorm()
    
    print("\n" + "=" * 60 + "\n")
    
    # Test inference mode  
    inference_result = test_inference_mode_batchnorm()
    
    print("=" * 60)
    
    # Save reference data for Fortran comparison
    save_reference_data(training_result, inference_result)
    
    print()
    print("🎯 KEY FINDINGS:")
    print("   1. PyTorch uses BIASED variance in training")
    print("   2. Momentum formula: new_running = (1-momentum)*old_running + momentum*batch")
    print("   3. Running statistics are NOT updated in eval mode")
    print("   4. Default eps=1e-05, momentum=0.1")
    print()
    print("✅ Compare these results with your Fortran implementation!")

if __name__ == '__main__':
    main()

🚀 Python cuDNN BatchNorm Reference Implementation

🔬 Testing BatchNorm in TRAINING mode
📊 Input shape: torch.Size([128, 32, 32, 32])
📊 Input mean: -0.000438
📊 Input std: 1.000164

📋 INITIAL BatchNorm Parameters:
   Weight (gamma) - first 5: [1. 1. 1. 1. 1.]
   Bias (beta) - first 5: [0. 0. 0. 0. 0.]
   Running mean - first 5: [0. 0. 0. 0. 0.]
   Running var - first 5: [1. 1. 1. 1. 1.]
   Momentum: 0.1
   Epsilon: 1e-05

🔄 Performing forward pass...
📊 Output shape: torch.Size([128, 32, 32, 32])
📊 Output mean: -0.000000
📊 Output std: 0.999995

📋 UPDATED BatchNorm Parameters (after training forward pass):
   Running mean - first 5: [-2.18730528e-04 -4.03176960e-04  1.52532919e-04  3.58769088e-04
  5.86154079e-06]
   Running var - first 5: [0.99982336 1.00089806 0.9994461  0.99983228 0.99973512]

📋 COMPUTED Batch Statistics (for verification):
   Batch mean - first 5: [-2.18730528e-03 -4.03176960e-03  1.52532919e-03  3.58769088e-03
  5.86154079e-05]
   Batch var - first 5: [0.99822603 1.00